In [ ]:
import pandas as pd

Using queries3.sql

1 user action per day up to 5x to get the full payment:

- US, UK, IE, DE, ES, IT: 8.75 USD (5 Daily User Actions to Max Partner Payment per User per month)
- BR: 7.50 USD (5 Daily User Actions to Max Partner Payment per User per month)
- MX, CL, AR: $6.25 USD (5 Daily User Actions to Max Partner Payment per User per month)

Day 1 (any user = 1.75$), then throughout the month, they would need to open the app 4 separate days to get full payment

Wellhub Data => https://docs.google.com/spreadsheets/d/1YsM0tvf2GOGXO6OcmKPjiMCPYRSpr7r3aXz5494AM-0/edit?gid=1426670745#gid=1426670745

### Importing data

In [ ]:
OUTPUT_DIR = "/Users/emulie/Data/Wellhub/output"

In [ ]:
df_internals = pd.read_csv("/Users/emulie/Data/Wellhub/wellhub_internals_20250327.csv")
# df_wellhub = pd.read_csv("/Users/emulie/Data/Wellhub/wellhub_users.csv")

In [ ]:
WELLHUB_FILE = "/Users/emulie/Data/Wellhub/wellhub_data_Jan2025_March2025.xlsx"
df_january = pd.read_excel(WELLHUB_FILE, sheet_name='january', engine='openpyxl')
df_february = pd.read_excel(WELLHUB_FILE, sheet_name='february', engine='openpyxl')
df_march = pd.read_excel(WELLHUB_FILE, sheet_name='march 1-15', engine='openpyxl')

In [ ]:
print(df_january.shape, df_february.shape, df_march.shape)

In [ ]:
df_internals.head()

In [ ]:
print(df_internals.shape)

### Counting monthly record per user 

In [ ]:
GROUPBY_COLS = ['gpw_id']
t = df_february.groupby(GROUPBY_COLS).count()

In [ ]:
t.shape

In [ ]:
# --- find all unique users
users_jan = df_january[['Distinct ID', 'gpw_id']].drop_duplicates()
users_feb = df_february[['Distinct ID', 'gpw_id']].drop_duplicates()
users_march = df_march[['Distinct ID', 'gpw_id']].drop_duplicates()

df_users = pd.concat([users_jan, users_feb, users_march], axis=0).drop_duplicates()

In [ ]:
print(df_users.shape)

In [ ]:
# --- is distinct ID user_id? Yes
has_used_bs_mask = df_users['Distinct ID'].isin(df_internals['user_id'].unique())

In [ ]:
df_month = df_users[has_used_bs_mask][['Distinct ID', 'gpw_id']].drop_duplicates()

In [ ]:
df_merged = pd.merge(df_month, df_internals, left_on='Distinct ID', right_on='user_id', how='left')

In [ ]:
df_merged

In [ ]:
# --- 
df_final = df_merged.groupby(['gpw_id', 'email']).sum()[['jan2025', 'feb2025', 'march2025']].reset_index()

In [ ]:
df_final

In [ ]:
df_final.to_csv(f"{OUTPUT_DIR}/wellhub_usage_jan2025_march2025.csv")

### Merging data